<a href="https://colab.research.google.com/github/jsebastianquiroga/Topicos_analitica_NLP/blob/main/Notebook/booksummaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Tópicos avanzados en analítica - NLP.</strong></h1> 
<h2> Para: Jaime Andrés Vargas Cruz.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Andres Guillermo Molano.</br></h2>
<h2>Nicolas Rodriguez Suarez.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>
<h2>Leonardo H. García.</br></h2>

In [2]:
import numpy as np
import pandas as pd
import os
import re
import csv
import json
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from collections import Counter


# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth

from oauth2client.client import GoogleCredentials

In [10]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Booksummaries_share = '1xU2UNslewl5MOZbXSff6XEjA_odaX6rv' 
download = drive.CreateFile({'id': Booksummaries_share}) 
download.GetContentFile('booksummaries.txt')


df_book_summaries = pd.read_csv("booksummaries.txt", header=None, sep="\t", names=['Wiki_ID', 'Freebase_ID', 'Title', 'Author', 'Pub_date', 'Genres', 'Summary'])

df_book_summaries

,Wiki_ID,Freebase_ID,Title,Author,Pub_date,Genres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [ ]:
# Mostrar información sobre el DataFrame
df_book_summaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Wiki_ID      16559 non-null  int64 
 1   Freebase_ID  16559 non-null  object
 2   Title        16559 non-null  object
 3   Author       14177 non-null  object
 4   Pub_date     10949 non-null  object
 5   Genres       12841 non-null  object
 6   Summary      16559 non-null  object
dtypes: int64(1), object(6)
memory usage: 905.7+ KB


Se puede identificar desde la perspectiva de procesamiento de lenguaje natural, las variables como el titulo, genero y summary podria traer información en cada una de sus corporas que nos ayuden a 

In [9]:
# Crear un DataFrame con el número de valores faltantes en cada columna
missing_values_df = pd.DataFrame(df_book_summaries.isna().sum(), columns=['recuento_faltantes'])

# Crear un gráfico de barras del recuento de valores faltantes utilizando Plotly
fig = px.bar(missing_values_df, x=missing_values_df.index, y='recuento_faltantes', 
             title='Valores faltantes en la base de datos')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Columnas', yaxis_title='Recuento de valores faltantes', title_x=0.5)

# Mostrar el gráfico
fig.show()

In [ ]:
# Convertir el género del formato json al formato de lista
def parse_genres(genre_str):
    try:
        return list(json.loads(genre_str).values())
    except (TypeError, json.JSONDecodeError):
        return []

# Aplicar la función parse_genres a la columna 'Genres'
df_book_summaries['Genres'] = df_book_summaries['Genres'].map(parse_genres)

# Mostrar las primeras 5 filas del DataFrame
df_book_summaries.head(5)

,Wiki_ID,Freebase_ID,Title,Author,Pub_date,Genres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,[],The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...


In [14]:
# Encontrar los autores más comunes
author_counts = df_book_summaries['Author'].value_counts().reset_index()
author_counts.columns = ['Autor', 'Recuento']

# Seleccionar los 10 principales autores con la cuenta más alta
top_authors = author_counts.nlargest(columns=["Recuento"], n=10)

# Restablecer el índice del DataFrame top_authors
top_authors.reset_index(drop=True, inplace=True)

# Ordenar los autores por Recuento
top_authors = top_authors.sort_values(by='Recuento', ascending=True)

# Crear un gráfico de barras de los recuentos de autor utilizando Plotly
fig = px.bar(top_authors, x='Recuento', y='Autor', 
             height=600, width=800, title='Autores más comunes')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Recuento', yaxis_title='Autor', 
                  title_x=0.5, title_font_size=24)

# Mostrar el gráfico
fig.show()

# Calcular el porcentaje de libros con un autor asociado
total_books = len(df_book_summaries)
books_with_author = df_book_summaries['Author'].notna().sum()
percentage_books_with_author = (books_with_author / total_books) * 100

# # Imprimir los resultados
# print("Autores más comunes:")
# print(author_counts.head(10))
print(f"\nPorcentaje de libros con un autor asociado: {percentage_books_with_author:.2f}%")


Porcentaje de libros con un autor asociado: 85.62%


In [23]:
# Convertir la columna 'Pub_date' al formato datetime
df_book_summaries['Pub_date'] = pd.to_datetime(df_book_summaries['Pub_date'], errors='coerce')

# Encontrar el periodo de tiempo cubierto por los libros analizados
earliest_date = df_book_summaries['Pub_date'].min()
latest_date = df_book_summaries['Pub_date'].max()

# Encontrar los libros más recientes y antiguos
most_recent_book = df_book_summaries.loc[df_book_summaries['Pub_date'].idxmax()]
oldest_book = df_book_summaries.loc[df_book_summaries['Pub_date'].idxmin()]

# Imprimir los resultados
print(f"Periodo de tiempo cubierto: {earliest_date} a {latest_date}")
print(f"Libro más reciente: {most_recent_book['Title']} (publicado el: {most_recent_book['Pub_date']})")
print(f"Libro más antiguo: {oldest_book['Title']} (publicado el: {oldest_book['Pub_date']})")

Periodo de tiempo cubierto: 1678-02-01 00:00:00 a 2013-01-01 00:00:00
Libro más reciente: Dr. Sleep (publicado el: 2013-01-01 00:00:00)
Libro más antiguo: The Pilgrim's Progress (publicado el: 1678-02-01 00:00:00)


In [25]:
# Encontrar el índice de los títulos más largos y más cortos
longest_title_index = df_book_summaries['Title'].apply(len).idxmax()
shortest_title_index = df_book_summaries['Title'].apply(len).idxmin()

# Obtener los títulos más largos y más cortos
longest_title = df_book_summaries.loc[longest_title_index, 'Title']
shortest_title = df_book_summaries.loc[shortest_title_index, 'Title']

# Imprimir los resultados
print(f"Título más largo: {longest_title} (longitud: {len(longest_title)})")
print(f"Título más corto: {shortest_title} (longitud: {len(shortest_title)})")

Título más largo: Passionate Minds: The Great Love Affair of the Enlightenment, Featuring the Scientist Emilie du Chatelet, the Poet Voltaire, Sword Fights, Book Burnings, Assorted Kings, Seditious Verse, and the Birth of the Modern World (longitud: 220)
Título más corto: Q (longitud: 1)


In [30]:
df_book_summaries['Genres'].astype(str)

0        {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1        {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2        {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
3                                                      nan
4        {"/m/03lrw": "Hard science fiction", "/m/06n90...
                               ...                        
16554                                                  nan
16555     {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction"}
16556                         {"/m/0xdf": "Autobiography"}
16557                                                  nan
16558    {"/m/02ql9": "Epistolary novel", "/m/014dfn": ...
Name: Genres, Length: 16559, dtype: object

In [31]:
# Contar los géneros individuales
genre_counts = Counter()
for genre_list in df_book_summaries['Genres']:
    genre_counts.update(genre_list)

# Crear un DataFrame con las columnas 'Género' y 'Recuento'
genre_count_df = pd.DataFrame({"Género": list(genre_counts.keys()), "Recuento": list(genre_counts.values())})

# Seleccionar los 10 principales géneros con la cuenta más alta
top_genres = genre_count_df.nlargest(columns=["Recuento"], n=10)

# Restablecer el índice del DataFrame top_genres
top_genres.reset_index(drop=True, inplace=True)

# Ordenar los géneros por Recuento
top_genres = top_genres.sort_values(by='Recuento', ascending=True)

# Crear un gráfico de barras de los recuentos de género utilizando Plotly
fig = px.bar(top_genres, x='Recuento', y='Género', 
             title='Géneros más comunes', height=600, width=800)

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Recuento', yaxis_title='Género', 
                  title_x=0.5, title_font_size=24, 
                  uniformtext_minsize=12, uniformtext_mode='hide')

# Mostrar el gráfico
fig.show()

# # Imprimir los recuentos de los 10 géneros principales utilizando el formato dado
# for index, row in top_genres.iterrows():
#     print(f"Género: {row['Genre']} (recuento: {row['Counts']})")

TypeError: ignored

In [ ]:
# # Eliminar las filas donde "Summary" y "Genres" son NaN
# df = df.dropna(subset=['Genres', 'Summary'])